# MLZoomcamp 2022 - Session #9 - Homework

Author: José Victor

* Dataset: [Dino or Dragon?](https://www.kaggle.com/datasets/agrigorev/dino-or-dragon)

* Model: [Model](https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5)

## Imports

In [1]:
import numpy as np
import tensorflow.lite as tflite

from tensorflow import keras

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the converted model?

* ( ) 21 Mb
* (X) 43 Mb
* ( ) 80 Mb
* ( ) 164 Mb

In [2]:
model = keras.models.load_model("dino_dragon_10_0.899.h5")

converter = tflite.TFLiteConverter.from_keras_model(model=model)

tflite_model = converter.convert()

with open("dino_dragon.tflite", "wb") as file:
    
    file.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\josev\AppData\Local\Temp\tmpbr23pgib\assets


INFO:tensorflow:Assets written to: C:\Users\josev\AppData\Local\Temp\tmpbr23pgib\assets


## Question 2

To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?

* ( ) 3
* ( ) 7
* (X) 13
* ( ) 24

In [3]:
interpreter = tflite.Interpreter(model_path="dino_dragon.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(output_index)

13


## Preparing the image

You'll need some code for downloading and resizing images. You can use this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```
pip install pillow
```

Let's download and resize this image:

[https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg](https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg)

Based on the previous homework, what should be the target size for the image?

In [18]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [5]:
image = download_image(url="https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg")
resized_image = prepare_image(img=image, target_size=(150, 150))

C:\Users\josev\AppData\Local\Temp\ipykernel_23716\2487457277.py:17: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


## Question 3

Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* ( ) 0.3353411
* (X) 0.5529412
* ( ) 0.7458824
* ( ) 0.9654902

In [8]:
x = np.array(resized_image)
x = x/255
X = np.array([x])

In [12]:
X[:,:,0][0][0][0]

0.5529411764705883

## Question 4

Now let's apply this model to this image. What's the output of the model?

* ( ) 0.17049132
* ( ) 0.39009996
* ( ) 0.60146144
* (X) 0.82448614

In [13]:
model.predict(X)

1/1 [==============================] - 0s 221ms/step


array([[0.82448477]], dtype=float32)

## Prepare the lambda code

Now you need to copy all the code into a separate python file. You will need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook by importing the file and invoking the function from this file.

## Docker

For the next two questions, we'll use a Docker image that we already prepared. This is the Dockerfile that we used for creating the image:

```
FROM public.ecr.aws/lambda/python:3.9
COPY dino-vs-dragon-v2.tflite .
```

And pushed it to `svizor42/zoomcamp-dino-dragon-lambda:v2`.

Note: The image already contains a model and it's not the same model as the one we used for questions 1-4.

## Question 5 

Download the base image `svizor42/zoomcamp-dino-dragon-lambda:v2`. You can easily make it by using `docker pull` command.

So what's the size of this base image?

* ( ) 139 Mb
* ( ) 329 Mb
* (X) 639 Mb
* ( ) 929 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [15]:
!docker images

REPOSITORY                             TAG       IMAGE ID       CREATED      SIZE
svizor42/zoomcamp-dino-dragon-lambda   v2        20ef58b21a05   8 days ago   639MB


## Question 6

Now let's extend this docker image, install all the required libraries and add the code for lambda.

You don't need to include the model in the image. It's already included. The name of the file with the model is `dino-vs-dragon-v2.tflite` and it's in the current workdir in the image (see the Dockerfile above for the reference).

Now run the container locally.

Score this image: [https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg](https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg)

What's the output from the model?

* ( ) 0.12
* (X) 0.32
* ( ) 0.52
* ( ) 0.72

In [19]:
!python .\test.py

0.31950658559799194


## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.

## Publishing to Docker hub

This is just for reference, this is how we published our image to Docker hub:

```
docker build -t zoomcamp-dino-dragon-lambda .
docker tag zoomcap-dino-dragon-lambda:latest svizor42/zoomcamp-dino-dragon-lambda:v2
docker push svizor42/zoomcamp-dino-dragon-lambda:v2
```